In [1]:
import tensorflow as tf

tf.__version__

'2.10.0'

In [ ]:
# Deskripsi dataset

# Folder dataset berisi 3 subfolder
# 1. training_set : berisi 2 subfolder yaitu cats dan dogs, masin-masing berisi 4000 gambar kucing dan anjing
# 2. test_set : berisi 2 subfolder yaitu cats dan dogs, masing-masing berisi 1000 gambar kucing dan anjing
# 3. testing : berisi 1 gambar kucing dan 1 gambar anjing

In [4]:
# Step 1 Data Preprocessing / Augmentation

# Import Dataset
from keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator adalah kelas untuk mengolah data gambar agar dapat digunakan untuk training model
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

# untuk memuat data, digunakan flow_from_directory method dari ImageDataGenerator
# method ini akan membaca gambar pada suatu direktory dan mengelompokkan gambar sesuai dengan subfolder
training_set = train_datagen.flow_from_directory(
    'image_dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(
    'image_dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
# Step 2 Membangun Model CNN

# Inisialisasi CNN
import keras

cnn = keras.models.Sequential()  # Inisiasi
cnn.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))  # 1st Convolutional Layer
# filters : jumlah feature map yang dihasilkan (biasanya 32)
# kernel_size : ukuran filter, dinyatakan dalam nxn matrix (biasanya 3x3)
# activation : fungsi aktivasi yang digunakan (biasanya relu)
# input_shape : ukuran gambar yang diinputkan ke CNN, dinyatakan dalam pxpxc matrix (biasanya 64x64x3)

cnn.add(keras.layers.MaxPool2D(pool_size=2, strides=2))  # 1st Pooling Layer
# pool_size : ukuran filter pooling, dinyatakan dalam nxn matrix (biasanya 2x2)
# strides : langkah yang digunakan filter pooling untuk bergerak (biasanya 2)

cnn.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))  # 2nd Convolutional Layer
cnn.add(keras.layers.MaxPool2D(pool_size=2, strides=2))  # 2nd Pooling Layer
cnn.add(keras.layers.Flatten())  # Flatten Layer
cnn.add(keras.layers.Dense(units=128, activation='relu'))  # Fully Connected Layer/Hidden Layer
# units : jumlah neuron dalam layer (biasanya 128)
# activation : fungsi aktivasi yang digunakan (biasanya relu)
cnn.add(keras.layers.Dense(units=1, activation='sigmoid'))  # Output Layer
# units : jumlah neuron dalam layer, tergantung pada jumlah kelas yang dihasilkan (binary classification = 1)

cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [10]:
# Step 3 Training Model CNN

# Compiling CNN

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# optimizer : algoritma yang digunakan untuk mengoptimalkan model (biasanya adam)
# loss : fungsi loss yang digunakan (binary classification = binary_crossentropy)
# metrics : metrik yang digunakan untuk mengevaluasi model (biasanya accuracy)

# Training CNN
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
# x : data training (ImageDataGenerator object yang sudah memiliki diisi dengan direktori)
# validation_data : data test (ImageDataGenerator object yang sudah memiliki diisi dengan direktori)
# epochs : jumlah iterasi training

Epoch 1/25
250/250 [==============================] - 122s 486ms/step - loss: 0.6560 - accuracy: 0.6015 - val_loss: 0.6167 - val_accuracy: 0.6695
Epoch 2/25
250/250 [==============================] - 96s 384ms/step - loss: 0.5910 - accuracy: 0.6891 - val_loss: 0.5484 - val_accuracy: 0.7240
Epoch 3/25
250/250 [==============================] - 48s 190ms/step - loss: 0.5435 - accuracy: 0.7219 - val_loss: 0.5167 - val_accuracy: 0.7500
Epoch 4/25
250/250 [==============================] - 50s 200ms/step - loss: 0.5103 - accuracy: 0.7449 - val_loss: 0.5791 - val_accuracy: 0.7210
Epoch 5/25
250/250 [==============================] - 55s 218ms/step - loss: 0.4863 - accuracy: 0.7644 - val_loss: 0.4879 - val_accuracy: 0.7695
Epoch 6/25
250/250 [==============================] - 51s 203ms/step - loss: 0.4676 - accuracy: 0.7741 - val_loss: 0.4632 - val_accuracy: 0.7860
Epoch 7/25
250/250 [==============================] - 53s 214ms/step - loss: 0.4473 - accuracy: 0.7884 - val_loss: 0.4963 - val_a

In [ ]:
# Step 4 Membuat Prediksi

from keras.preprocessing import image
import numpy as np

test_image = image.load_img('dataset/testing/cat_or_dog_1.jpg', target_size = (64, 64)) # Single image loader
test_image = image.img_to_array(test_image) # Convert image to array
# model menerima input berupa array, maka perlu diubah dari image ke array
# model menerima input array 3D 64x64x32, 64x64 adalah ukuran gambar, 32 adalah jumlah batch
# oleh karena itu, input untuk prediksi perlu diekspan menjadi 3D

test_image = np.expand_dims(test_image, axis = 0) # Expand dimension

result = cnn.predict(test_image)

training_set.class_indices

# printing results
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)